In [1]:
using DataDrivenDiffEq
using ModelingToolkit
using OrdinaryDiffEq

using LinearAlgebra
using Plots
gr()

Plots.GRBackend()

In [2]:
# Create a test problem
function lorenz(u,p,t)
    x, y, z = u
    ẋ = 10.0*(y - x)
    ẏ = x*(28.0-z) - y
    ż = x*y - (8/3)*z
    return [ẋ, ẏ, ż]
end

u0 = [-8.0; 7.0; 27.0]
p = [10.0; -10.0; 28.0; -1.0; -1.0; 1.0; -8/3]
tspan = (0.0,100.0)
dt = 0.001
problem = ODEProblem(lorenz,u0,tspan)
solution = solve(problem, Tsit5(), saveat = dt, atol = 1e-7, rtol = 1e-8)

X = Array(solution)
DX = similar(X)
for (i, xi) in enumerate(eachcol(X))
    DX[:,i] = lorenz(xi, [], 0.0)
end

In [3]:
DX

3×100001 Array{Float64,2}:
  150.0   148.354    146.717    145.088   …  -27.407   -27.6211  -27.8369
  -15.0   -15.8344   -16.6204   -17.3596     -48.7382  -49.1132  -49.4897
 -128.0  -126.495   -125.005   -123.529      -10.7166  -10.3061   -9.89088

In [4]:
## Now automatically discover the system that generated the data
################################################################

@variables x y z
u = [x; y; z]
polys = Operation[]
for i ∈ 0:4
    for j ∈ 0:i
        for k ∈ 0:j
            push!(polys, u[1]^i*u[2]^j*u[3]^k)
            push!(polys, u[2]^i*u[3]^j*u[1]^k)
            push!(polys, u[3]^i*u[1]^j*u[2]^k)
        end
    end
end

basis = Basis(polys, u)

opt = STRRidge(0.1)
Ψ = SInDy(X, DX, basis, opt, maxiter = 100, normalize = true)
print_equations(Ψ)
get_error(Ψ)

MethodError: MethodError: no method matching SInDy(::Array{Float64,2}, ::Array{Float64,2}, ::Basis{Array{Expression,1},Array{Operation,1},Array{Operation,1},Operation}, ::STRRidge{Float64}; maxiter=100, normalize=true)
Closest candidates are:
  SInDy(::AbstractArray{S,2}, ::AbstractArray{S,2}, ::Basis; p, t, maxiter, opt, denoise, normalize, convergence_error) where S<:Number at C:\Users\valpe\.julia\packages\DataDrivenDiffEq\kn8bK\src\sindy\sindy.jl:144
  SInDy(::Any, ::Any, ::Basis, !Matched::AbstractArray; f, g, p, t, maxiter, opt, denoise, normalize, convergence_error) at C:\Users\valpe\.julia\packages\DataDrivenDiffEq\kn8bK\src\sindy\sindy.jl:160
  SInDy(!Matched::AbstractArray{S,1}, ::AbstractArray, ::Basis; kwargs...) where S<:Number at C:\Users\valpe\.julia\packages\DataDrivenDiffEq\kn8bK\src\sindy\sindy.jl:117
  ...